In [1]:
from collections import OrderedDict

import numpy as np
from qiskit import Aer
from qiskit.transpiler import PassManager

from qiskit_aqua import Operator, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.algorithms.components.optimizers import L_BFGS_B
from qiskit_aqua.algorithms.components.variational_forms import RY

from qiskit_chemistry import FermionicOperator
from qiskit_chemistry.drivers import ConfigurationManager

In [2]:
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'H .0 .0 .0; H .0 .0 0.735'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), \
                         ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_jw.chop(10**-10)

# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(qubitOp_jw.print_operators())

print('The exact ground state energy is: {}'.format(ret['energy']))
print('The Hartree Fock Electron Energy is: {}'.format(molecule.hf_energy - molecule.nuclear_repulsion_energy))

The exact ground state energy is: -1.8572750302023824
The Hartree Fock Electron Energy is: -1.8369679912029842


In [4]:
# particle hole transformation
newferOp, energy_shift = ferOp.particle_hole_transformation(num_particles=2)
print('Energy shift is: {}'.format(energy_shift))
newqubitOp_jw = newferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
newqubitOp_jw.chop(10**-10)

exact_eigensolver = ExactEigensolver(newqubitOp_jw, k=1)
ret = exact_eigensolver.run()

# print(newqubitOp_jw.print_operators())
print('The exact ground state energy in PH basis is {}'.format(ret['energy']))
print('The exact ground state energy in PH basis is {} (with energy_shift)'.format(ret['energy'] - energy_shift))

Energy shift is: 1.8369679912029846
The exact ground state energy in PH basis is -0.020307038999395587
The exact ground state energy in PH basis is -1.8572750302023802 (with energy_shift)


In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B()
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)

# setup variation form generator (generate trial circuits for VQE)
var_form = RY(newqubitOp_jw.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = VQE(newqubitOp_jw, var_form, lbfgs, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Minimum value: {}".format(results['eigvals'][0].real - energy_shift))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -0.020307038916806994
Minimum value: -1.8572750301197916
Parameters: [-0.61946199 -1.87409724  0.24417792  1.60641243 -2.3985038  -2.55902994
 -1.68929972 -0.55469298 -1.09966074 -3.00271563  2.53835355  0.30461019
  0.86787481  2.57016619 -2.17943234 -1.08788746  2.60613503  2.21836476
 -1.77645011 -1.88678767  3.14158727  0.69993303  1.62197744 -0.65831574]
